# PPYOLOE
PP-YOLOE是基于PP-YOLOv2的卓越的单阶段Anchor-free模型，超越了多种流行的YOLO模型。PP-YOLOE有一系列的模型，即s/m/l/x，可以通过width multiplier和depth multiplier配置。PP-YOLOE避免了使用诸如Deformable Convolution或者Matrix NMS之类的特殊算子，以使其能轻松地部署在多种多样的硬件上。

根据PaddleDetection给出的云端模型性能对比，各模型结构和骨干网络的代表模型在COCO数据集上精度mAP和单卡Tesla V100上预测速度(FPS)对比图如下：

![https://gitee.com/paddlepaddle/PaddleDetection/raw/develop/docs/images/fps_map.png](https://gitee.com/paddlepaddle/PaddleDetection/raw/develop/docs/images/fps_map.png)

可以看出，PP-YOLOE真可谓是【又快又好】的典型！

这还不够，PaddleDetection团队还提供了基于PP-YOLOE的各种垂类检测模型的配置文件和权重，供用户下载进行使用：

| 场景 | 相关数据集 | 链接 |
| :-: | :-: | :-: |
| 行人检测 | CrowdHuman | [pphuman](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/configs/pphuman) |
| 车辆检测 | BDD100K、UA-DETRAC | [ppvehicle](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/configs/ppvehicle) |
| 小目标检测 | VisDrone | [visdrone](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/configs/visdrone) |

本项目参考了**PPYOLOE：又快又好的小目标检测训练与部署实现**

https://aistudio.baidu.com/aistudio/projectdetail/4435291?channelType=0&channel=0

# 1 环境准备
## 1.1 数据集准备
数据集的分析和准备过程可参考博客
[使用PPYOLOE训练目标检测](http://)


1.**导入所需要的第三方库**

In [3]:
# 调用一些需要的第三方库
import numpy as np
import pandas as pd
import shutil
import json
import os
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.font_manager import FontProperties
from PIL import Image
import random
myfont = FontProperties(fname=r"NotoSansCJKsc-Medium.otf", size=12)
plt.rcParams['figure.figsize'] = (12, 12)
plt.rcParams['font.family']= myfont.get_family()
plt.rcParams['font.sans-serif'] = myfont.get_name()
plt.rcParams['axes.unicode_minus'] = False

2.**安装paddlex**

In [4]:
# 引入PaddleX
!pip install paddlex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593874 sha256=5325d4a64186f6dacb6ae5aa168f5ae2c2894f4451452e7a6ea96f249bc92a49
  Stored in directory: /home/aistudio/.cache/pip/wheels/5c/d0/d2/e331d17a999666b1e2eb99743cfa1742629f9d26c55c657001
  Created wheel for pycocotools: filename=pycoc

3.**创建数据集目录** 

将标注的图像数据上传到 **MyDataset/JPEGImages** 目录下。

将coco格式数据标签**annotations.json**放到**MyDataset**目录下。

In [5]:
# 组织数据目录
!mkdir MyDataset
!mkdir MyDataset/JPEGImages

4.**按比例切分数据集**

In [6]:
# 按比例切分数据集
!paddlex --split_dataset --format COCO --dataset_dir /home/aistudio/MyDataset --val_value 0.1 --test_value 0.0

[03-20 14:46:06 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format COCO --dataset_dir /home/aistudio/MyDataset --val_value 0.1 --test_value 0.0
[03-20 14:46:06 MainThread @utils.py:79] WRN paddlepaddle version: 2.3.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-p

## 1.2 训练环境准备

由于PP-YOLOE还在快速迭代中，因此，对框架的稳定性有一定的要求，PaddlePaddle的框架不要选择最新版。本文使用的单卡训练环境如下：

- 框架版本：PaddlePaddle 2.2.2
- CUDA Version: 11.2
- 模型库版本：PaddleDetection(develop分支)

选择PaddleDetection(develop分支)的原因是，PP-YOLOE的垂类模型迭代更快些，选择空间更大。

5. **git PaddleDetection**代码

In [7]:
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git

正克隆到 'PaddleDetection'...
remote: Enumerating objects: 256938, done.
remote: Counting objects: 100% (1935/1935), done.
remote: Compressing objects: 100% (1131/1131), done.
remote: Total 256938 (delta 1187), reused 1465 (delta 798), pack-reused 255003
接收对象中: 100% (256938/256938), 413.52 MiB | 6.11 MiB/s, 完成.
处理 delta 中: 100% (210215/210215), 完成.
检查连接... 完成。


6. **进入PaddleDetection目录**

In [8]:
%cd PaddleDetection
# 切换到develop分支
!git checkout develop

/home/aistudio/PaddleDetection
分支 develop 设置为跟踪来自 origin 的远程分支 develop。
切换到一个新分支 'develop'


# 2 模型训练
## 2.1 模型选型

PaddleDetection团队提供了针对VisDrone-DET小目标数航拍场景的基于PP-YOLOE的检测模型，本项目使用这些模型作为预训练模型。

本项目以`PP-YOLOE-Alpha-largesize-l`为例，展示训练和部署过程。


## 2.2 模型训练

7. **根据需求修改配置文件，比如检测的目标类别数**

进入/home/aistudio/config_file/目录下，修改visdrone_detection.yml中**num_classes**参数


In [9]:
# 训练配置文件覆盖
!cd configs
!mkdir configs/visdrone/
!cd ../
!cp ../config_file/visdrone/ppyoloe_crn_l_80e_visdrone.yml configs/visdrone/ppyoloe_crn_l_80e_visdrone.yml
!cp ../config_file/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml configs/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml
!cp ../config_file/visdrone_detection.yml configs/datasets/visdrone_detection.yml
!cp ../config_file/optimizer_300e.yml configs/ppyoloe/_base_/optimizer_300e.yml
!cp ../config_file/ppyoloe_crn.yml configs/ppyoloe/_base_/ppyoloe_crn.yml
!cp ../config_file/ppyoloe_reader.yml configs/ppyoloe/_base_/ppyoloe_reader.yml

8. **开始训练**

mAP 不再增长时，即可停止训练。

In [11]:
# 开始训练
!python tools/train.py -c configs/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml --use_vdl=True --vdl_log_dir=./visdrone/ --eval

^C
Error: Can not import avx core while this file exists: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/core_avx.so
Traceback (most recent call last):
  File "tools/train.py", line 30, in <module>
    import paddle
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/__init__.py", line 25, in <module>
    from .framework import monkey_patch_variable
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/framework/__init__.py", line 17, in <module>
    from . import random  # noqa: F401
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/framework/random.py", line 16, in <module>
    import paddle.fluid as fluid
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/__init__.py", line 36, in <module>
    from . import framework
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py", line 37

9. **训练完成后评估模型**

In [12]:
# 模型评估
!python tools/eval.py -c configs/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml -o weights=output/ppyoloe_crn_l_alpha_largesize_80e_visdrone/best_model.pdparams

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
W0320 14:52:46.793298  8603 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0320

## 2.3 预测推理

In [13]:
# 挑一张验证集的图片展示预测效果
!python tools/infer.py -c configs/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml -o weights=output/ppyoloe_crn_l_alpha_largesize_80e_visdrone/best_model --infer_img=/home/aistudio/MyDataset/JPEGImages/1679146167842.jpg --save_results=True

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
W0320 14:53:01.727855  8750 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0320

# 3 模型部署
同样是小目标检测，相比于`SNIPER: Efficient Multi-Scale Training`暂不支持部署，PP-YOLOE是可以直接导出部署模型，并在多端高性能部署的：
- Paddle Inference [Python](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/deploy/python) & [C++](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/deploy/cpp)
- [Paddle-TensorRT](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/deploy/TENSOR_RT.md)
- [PaddleServing](https://gitee.com/link?target=https%3A%2F%2Fgithub.com%2FPaddlePaddle%2FServing)
- [PaddleSlim模型量化](https://gitee.com/paddlepaddle/PaddleDetection/blob/develop/configs/slim)

接下来，我们将介绍PP-YOLOE如何使用Paddle Inference进行部署。
## 3.1 导出模型


10. **导出模型，即可使用FastDeploy进行快速推理**

https://github.com/PaddlePaddle/FastDeploy

In [14]:
!python tools/export_model.py -c configs/visdrone/ppyoloe_crn_l_alpha_largesize_80e_visdrone.yml -o weights=output/ppyoloe_crn_l_alpha_largesize_80e_visdrone/best_model.pdparams

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
[03/20 14:53:43] ppdet.utils.checkpoint INFO: Finish loading model weights: output/ppyoloe_crn_l_alpha_largesize_80e_visdrone/best_model.pdparams
loading annot